## Lab Assignment 2 [Text Analytics - CISB5123]

### <span style='background :yellow' > Student Information (Group Members) </span>

Member 1

**Name     :** Zulfadhli Fakhri bin Johan

**ID No    :** SW01081044

**Section  :** 02

=============================================================

Member 2

**Name     :** Errenbai Yeerhanati

**ID No    :** SW01081538

**Section  :** 02

In [1]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [4]:
# Load dataset
df = pd.read_csv("Reviews.csv").head(20000)

In [5]:
# Display count of each score
print("Score Value Counts:")
print(df['Score'].value_counts())

Score Value Counts:
Score
5    12571
4     2836
1     1824
3     1649
2     1120
Name: count, dtype: int64


In [6]:
# Display first 5 rows BEFORE modifying the score
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [7]:
# Change the values of the Score column
df['Score'] = df['Score'].map({1: 0, 2: 0, 3: 1, 4: 2, 5: 2})

# Score 1 and 2 = negative (0)
# Score 3 = neutral (1)
# Score 4 and 5 = positive (2)

In [8]:
# Display count of updated score
print("Score Value Counts (Modified):")
print(df['Score'].value_counts())

Score Value Counts (Modified):
Score
2    15407
0     2944
1     1649
Name: count, dtype: int64


In [9]:
# Display first 5 rows AFTER modifying the score
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,2,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,2,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,2,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [10]:
# Data preprocessing
stop_words = stopwords.words('english')
more_stopwords = ['u', 'im', 'ive', 'would', 'havent', 'dont', 'id', 'etc', 'wasnt', 'didnt', 'theyll', 'oh', 'cant',
                 'doesnt', 'youre', 'youd', 'youll', 'theyre', 'whats', 'thats', 'isnt', 'wont', 'werent', 'weve',
                  'shouldve', 'couldnt', 'could']
stop_words = stop_words + more_stopwords
lemmatizer = WordNetLemmatizer()

In [11]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [12]:
from nltk.tokenize import word_tokenize

def preprocess(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'\[.*?\]', '', text)  # Remove text within square brackets
    text = re.sub(r'http\S+\s*\S+', '', text)  # Remove URLs starting with http
    text = re.sub(r'www\.\S+', '', text)  # Remove URLs starting with www
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\b\w*\d\w*\b', '', text)  # Remove words containing numbers
    tokens = word_tokenize(text) # Tokenize
    text = ' '.join(word for word in tokens if word not in stop_words) # Remove stopwords
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split(' ')) # Lemmatization
    return text

In [13]:
df['clean_text'] = df['Text'].apply(preprocess)

In [14]:
# Display first 5 rows AFTER preprocessed Text into clean_text
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,clean_text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,2,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,bought several vitality canned dog food produc...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,product arrived labeled jumbo salted peanutsth...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,2,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,confection around century light pillowy citrus...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,looking secret ingredient robitussin believe f...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,2,1350777600,Great taffy,Great taffy at a great price. There was a wid...,great taffy great price wide assortment yummy ...


In [15]:
# Save Text and cleaned_text into a new CSV file
df_text_clean = pd.DataFrame(data={'Actual Text': df['Text'], 'Cleaned Text': df['clean_text'], 'Actual Label': df['Score']})
df_text_clean.to_csv('Preprocessed_Text.csv', index=False)
df_text_clean.head()

,Actual Text,Cleaned Text,Actual Label
0,I have bought several of the Vitality canned d...,bought several vitality canned dog food produc...,2
1,Product arrived labeled as Jumbo Salted Peanut...,product arrived labeled jumbo salted peanutsth...,0
2,This is a confection that has been around a fe...,confection around century light pillowy citrus...,2
3,If you are looking for the secret ingredient i...,looking secret ingredient robitussin believe f...,0
4,Great taffy at a great price. There was a wid...,great taffy great price wide assortment yummy ...,2


In [16]:
# Feature Extraction using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df['clean_text'])
y = df['Score']

In [17]:
# Model Selection
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [18]:
# Multinomial Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
nb_pred = nb_model.predict(X_test)

In [19]:
# Support Vector Machine (SVM) model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)

In [20]:
# Model Evaluation
svm_accuracy = accuracy_score(y_test, svm_pred)
nb_accuracy = accuracy_score(y_test, nb_pred)

print("SVM Accuracy:", svm_accuracy)
print("Naive Bayes Accuracy:", nb_accuracy)

print("\nSVM Classification Report:")
print(classification_report(y_test, svm_pred, zero_division=1, target_names=['negative', 'neutral', 'positive']))

print("\nNaive Bayes Classification Report:")
print(classification_report(y_test, nb_pred, zero_division=1, target_names=['negative', 'neutral', 'positive']))

SVM Accuracy: 0.84825
Naive Bayes Accuracy: 0.7705

SVM Classification Report:
              precision    recall  f1-score   support

    negative       0.76      0.58      0.66       589
     neutral       0.61      0.08      0.14       330
    positive       0.86      0.98      0.92      3081

    accuracy                           0.85      4000
   macro avg       0.75      0.55      0.57      4000
weighted avg       0.83      0.85      0.82      4000


Naive Bayes Classification Report:
              precision    recall  f1-score   support

    negative       0.67      0.00      0.01       589
     neutral       1.00      0.00      0.00       330
    positive       0.77      1.00      0.87      3081

    accuracy                           0.77      4000
   macro avg       0.81      0.33      0.29      4000
weighted avg       0.77      0.77      0.67      4000



In [21]:
# Lexicon-based approach using TextBlob and VADER
table_data = [["Text", "Actual Label", "TextBlob Polarity", "TextBlob Sentiment", 
               "VADER Compound", "VADER Sentiment"]]

analyzer = SentimentIntensityAnalyzer()
for text, actual_label in zip(df['clean_text'], df['Score']):
    # TextBlob
    blob = TextBlob(text)
    tb_polarity = blob.sentiment.polarity

    # Determine label based on polarity score from TextBlob
    if tb_polarity > 0:
        tb_label = 2
    elif tb_polarity < 0:
        tb_label = 0
    else:
        tb_label = 1

    # VADER
    vs = analyzer.polarity_scores(text)
    vader_compound = vs['compound']

    # Determine label based on compound score from VADER
    if vader_compound > 0.05:
        vader_label = 2
    elif vader_compound < -0.05:
        vader_label = 0
    else:
        vader_label = 1

    table_data.append([text, actual_label, tb_polarity, tb_label, vader_compound, vader_label])

In [22]:
# Calculate classification report for TextBlob
tb_classification_report = classification_report([label for _, label, _, tb_label, _, _ in table_data[1:]], 
                                                  [tb_label for _, _, _, tb_label, _, _ in table_data[1:]], 
                                                  target_names=['negative', 'neutral', 'positive'])

# Print classification report for TextBlob
print("\nClassification Report for TextBlob:")
print(tb_classification_report)


Classification Report for TextBlob:
              precision    recall  f1-score   support

    negative       0.53      0.34      0.41      2944
     neutral       0.08      0.02      0.03      1649
    positive       0.82      0.94      0.88     15407

    accuracy                           0.78     20000
   macro avg       0.47      0.43      0.44     20000
weighted avg       0.71      0.78      0.74     20000



In [23]:
# Calculate classification report for VADER
vader_classification_report = classification_report([label for _, label, _, _, _, vader_label in table_data[1:]], 
                                                     [vader_label for _, _, _, _, _, vader_label in table_data[1:]], 
                                                     target_names=['negative', 'neutral', 'positive'])

# Print classification report for VADER
print("\nClassification Report for VADER:")
print(vader_classification_report)


Classification Report for VADER:
              precision    recall  f1-score   support

    negative       0.59      0.28      0.38      2944
     neutral       0.15      0.04      0.06      1649
    positive       0.81      0.96      0.88     15407

    accuracy                           0.78     20000
   macro avg       0.52      0.43      0.44     20000
weighted avg       0.73      0.78      0.74     20000



In [24]:
# Convert array of words into regular sentences
cleaned_text = [' '.join(words) for words in tfidf_vectorizer.inverse_transform(X_test)]

# Save X and predicted labels for Multinomial Naive Bayes model
df_nb_result = pd.DataFrame(data={'Cleaned Text': cleaned_text, 'Actual Label': y_test, 'NB Sentiment': nb_pred})
df_nb_result.to_csv('X_with_Predicted_Labels_NB.csv', index=False)
df_nb_result.head(10)

,Cleaned Text,Actual Label,NB Sentiment
2069,beat bisquick doubt expensive fake grocery le ...,0,2
14914,buying chip chocolate chunk cooky delicious ea...,2,2
7634,arrived came enjoy excellently good growing he...,2,2
17782,admit bag big buy day delicious end entire eve...,2,2
9327,buy delicious different excellent hassle one p...,2,2
3660,attributable barely better bitter blueberry bo...,0,2
19340,cherry customer enjoyed fl friend love plenty ...,2,2
712,absolutely actually addition although amazing ...,2,2
95,allergic also balance different dog food free ...,2,2
12194,added adulterant ago almost brand cheap chicke...,0,2


In [25]:
# Convert array of words into regular sentences
cleaned_text = [' '.join(words) for words in tfidf_vectorizer.inverse_transform(X_test)]

# Save X and predicted labels for Support Vector Machine (SVM) model
df_svm_result = pd.DataFrame(data={'Cleaned Text': cleaned_text, 'Actual Label': y_test, 'SVM Sentiment': svm_pred})
df_svm_result.to_csv('X_with_Predicted_Labels_SVM.csv', index=False)
df_svm_result.head(10)

,Cleaned Text,Actual Label,SVM Sentiment
2069,beat bisquick doubt expensive fake grocery le ...,0,2
14914,buying chip chocolate chunk cooky delicious ea...,2,2
7634,arrived came enjoy excellently good growing he...,2,2
17782,admit bag big buy day delicious end entire eve...,2,2
9327,buy delicious different excellent hassle one p...,2,2
3660,attributable barely better bitter blueberry bo...,0,0
19340,cherry customer enjoyed fl friend love plenty ...,2,2
712,absolutely actually addition although amazing ...,2,2
95,allergic also balance different dog food free ...,2,2
12194,added adulterant ago almost brand cheap chicke...,0,0


In [26]:
# Lexicon-based approach using TextBlob and VADER
df_lexicon = pd.DataFrame(table_data[1:], columns=table_data[0])
df_lexicon.to_csv('Reviews_Lexicon_Predictions.csv', index=False)
df_lexicon.head(10)

,Text,Actual Label,TextBlob Polarity,TextBlob Sentiment,VADER Compound,VADER Sentiment
0,bought several vitality canned dog food produc...,2,0.425000,2,0.9413,2
1,product arrived labeled jumbo salted peanutsth...,0,0.216667,2,-0.1027,0
2,confection around century light pillowy citrus...,2,0.187000,2,0.8532,2
3,looking secret ingredient robitussin believe f...,0,0.150000,2,0.4404,2
4,great taffy great price wide assortment yummy ...,2,0.458333,2,0.9468,2
5,got wild hair taffy ordered five pound bag taf...,2,0.333333,2,0.9136,2
6,saltwater taffy great flavor soft chewy candy ...,2,0.210000,2,0.9463,2
7,taffy good soft chewy flavor amazing definitel...,2,0.380000,2,0.9313,2
8,right mostly sprouting cat eat grass love rota...,2,0.428571,2,0.6369,2
9,healthy dog food good digestion also good smal...,2,0.412500,2,0.8176,2


### Discussion

Strengths and weaknesses of the selected models for sentiment classification:

=========================================================================================================

Support Vector Machine (SVM) model: 

Strengths:
- SVM performs well in high dimensional spaces.
- It is effective when the number of dimensions is greater than the number of samples.

Weaknesses:
- SVM can be computationally expensive, especially with large datasets.

=========================================================================================================

Multinomial Naive Bayes model:

Strengths:
- Naive Bayes is simple and fast.
- It works well with high-dimensional data like text data.

Weaknesses:
- Naive Bayes makes a strong assumption about the independence of features, which may not always be true in real-world data.

=========================================================================================================

TextBlob:

Strengths:
- TextBlob provides a simple API for common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, and sentiment analysis.
- It is easy to use and requires minimal preprocessing.

Weaknesses:
- TextBlob may not perform well on highly specialized or domain-specific text data.
- It relies on a pre-trained model and may not capture nuances in sentiment as effectively as custom-trained models.

=========================================================================================================

VADER (Valence Aware Dictionary and sEntiment Reasoner):

Strengths:
- VADER is specifically designed for sentiment analysis of social media text.
- It provides a score for the sentiment intensity of text, allowing for fine-grained analysis.

Weaknesses:
- VADER's performance may vary depending on the type of text being analyzed.
- It may not perform as well on formal or structured text data compared to informal or unstructured text.

=========================================================================================================l or structured text data compared to informal or unstructured text.
""" to informal or unstructured text.
"""
 informal or unstructured text.
"""
